# VishwamAI Training Performance Analysis

This notebook analyzes the training performance with TPU optimizations and fixed FlashAttention implementation.

In [ ]:
# Import required libraries
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any, Optional
from dataclasses import dataclass

from vishwamai.training import TPUTrainingConfig, create_train_state_tpu, create_train_step_tpu
from vishwamai.profiler import TPUProfiler
from vishwamai.transformer import create_vishwamai_transformer

In [ ]:
# Load training and test data
train_data = pd.read_parquet('/home/kasinadhsarma/VishwamAI/train-00000-of-00001.parquet')
test_data = pd.read_parquet('/home/kasinadhsarma/VishwamAI/test-00000-of-00001.parquet')

print("Training data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
@dataclass
class TrainingConfig:
    """Training configuration with proper typing"""
    model_config: Dict[str, Any]
    batch_size: int
    grad_accum_steps: int
    learning_rate: float
    warmup_steps: int
    max_steps: int
    weight_decay: float
    max_grad_norm: float
    dtype: str
    enable_pjit: bool
    block_size: int
    use_flash_attn: bool
    mixed_precision: bool

def create_training_config() -> TrainingConfig:
    """Create TPU-optimized training configuration"""
    model_config = {
        'vocab_size': 32000,
        'num_layers': 12,
        'num_heads': 12,
        'head_dim': 64,
        'hidden_dim': 768,
        'mlp_dim': 3072,
        'max_seq_len': 2048,
        'dropout_rate': 0.1,
        'use_flash_attn': True,
        'use_rotary': True,
        'use_rms_norm': False
    }
    
    return TrainingConfig(
        model_config=model_config,
        batch_size=32,
        grad_accum_steps=4,
        learning_rate=1e-4,
        warmup_steps=2000,
        max_steps=100000,
        weight_decay=0.01,
        max_grad_norm=1.0,
        dtype='bfloat16',
        enable_pjit=True,
        block_size=128,
        use_flash_attn=True,
        mixed_precision=True
    )

In [ ]:
# Initialize training components
print("Creating training configuration...")
config = create_training_config()

print("\nInitializing model and training state...")
rng = jax.random.PRNGKey(42)
state = create_train_state_tpu(TPUTrainingConfig(**vars(config)), rng)
train_step = create_train_step_tpu(TPUTrainingConfig(**vars(config)), state)

print("\nSetting up profiler...")
profiler = TPUProfiler(config=config.model_config)

In [ ]:
def analyze_training_performance(
    profiler: TPUProfiler,
    num_steps: int = 100,
    log_interval: int = 10
) -> Dict[str, list]:
    """Analyze training performance metrics with enhanced monitoring"""
    metrics = {
        'step_time': [],
        'throughput': [],
        'memory_used': [],
        'tpu_utilization': [],
        'compute_efficiency': [],
        'memory_efficiency': []
    }
    
    for step in range(num_steps):
        profiler.start_step()
        
        # Simulate training step
        batch_size = config.batch_size * config.grad_accum_steps
        profiler.record_batch_time(batch_size, 0.1)  # Example duration
        profiler.measure_tpu_utilization()
        
        profiler.end_step()
        
        # Collect detailed metrics
        summary = profiler.get_metrics_summary()
        metrics['step_time'].append(summary['step_time_mean'])
        metrics['throughput'].append(summary.get('steps_per_second', 0))
        metrics['memory_used'].append(summary.get('memory_accessed_mean', 0))
        metrics['tpu_utilization'].append(summary.get('tpu_utilization_mean', 0))
        
        # Calculate efficiency metrics
        compute_efficiency = summary.get('compute_time_mean', 0) / summary['step_time_mean']
        memory_efficiency = 1 - (summary.get('memory_stall_mean', 0) / summary['step_time_mean'])
        
        metrics['compute_efficiency'].append(compute_efficiency)
        metrics['memory_efficiency'].append(memory_efficiency)
        
        if step % log_interval == 0:
            print(f"\nStep {step}/{num_steps}")
            print(f"TPU Utilization: {metrics['tpu_utilization'][-1]:.2%}")
            print(f"Throughput: {metrics['throughput'][-1]:.2f} steps/sec")
            print(f"Memory Usage: {metrics['memory_used'][-1]/1e9:.2f} GB")
    
    return metrics

# Run analysis
print("\nAnalyzing training performance...")
training_metrics = analyze_training_performance(profiler)

# Create enhanced visualization
plt.style.use('seaborn')
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
fig.suptitle('VishwamAI Training Performance Analysis', fontsize=16)

# Plot metrics with enhanced styling
def plot_metric(ax, data, title, ylabel, color='#2196F3'):
    ax.plot(data, color=color, linewidth=2)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('Step', fontsize=10)
    ax.set_ylabel(ylabel, fontsize=10)
    ax.grid(True, alpha=0.3)

plot_metric(axes[0,0], training_metrics['step_time'], 'Step Time', 'Time (s)')
plot_metric(axes[0,1], training_metrics['throughput'], 'Training Throughput', 'Steps/Second', '#4CAF50')
plot_metric(axes[1,0], [m/1e9 for m in training_metrics['memory_used']], 'Memory Usage', 'GB', '#FF9800')
plot_metric(axes[1,1], training_metrics['tpu_utilization'], 'TPU Utilization', 'Utilization %', '#E91E63')
plot_metric(axes[2,0], training_metrics['compute_efficiency'], 'Compute Efficiency', 'Ratio', '#9C27B0')
plot_metric(axes[2,1], training_metrics['memory_efficiency'], 'Memory Efficiency', 'Ratio', '#795548')

plt.tight_layout()
plt.show()

# Generate performance insights
print("\nPerformance Analysis Summary:")
print(f"Average TPU Utilization: {np.mean(training_metrics['tpu_utilization']):.2%}")
print(f"Average Throughput: {np.mean(training_metrics['throughput']):.2f} steps/sec")
print(f"Peak Memory Usage: {max(training_metrics['memory_used'])/1e9:.2f} GB")
print(f"Average Compute Efficiency: {np.mean(training_metrics['compute_efficiency']):.2%}")
print(f"Average Memory Efficiency: {np.mean(training_metrics['memory_efficiency']):.2%}")

# Get and display optimization recommendations
recommendations = profiler.get_performance_recommendations()
print("\nPerformance Optimization Recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec}")

In [ ]:
# Create configuration with proper dtype settings
config = create_training_config()

# Initialize model and check params
print("Creating model...")
model = create_vishwamai_transformer(config)

# Initialize training components
print("\nInitializing training state...")
rng = jax.random.PRNGKey(42)
state = create_train_state_tpu(config, rng)
train_step = create_train_step_tpu(config, state)

# Initialize profiler with proper config
print("\nSetting up profiler...")
profiler = TPUProfiler(config=config['model_config'])

# Run analysis
print("\nAnalyzing training metrics...")
training_metrics = analyze_training_metrics(profiler)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Training Performance Analysis')

steps = range(len(training_metrics['step_time']))

axes[0,0].plot(steps, training_metrics['step_time'])
axes[0,0].set_title('Step Time')
axes[0,0].set_xlabel('Step')
axes[0,0].set_ylabel('Time (s)')

axes[0,1].plot(steps, training_metrics['throughput'])
axes[0,1].set_title('Training Throughput')
axes[0,1].set_xlabel('Step')
axes[0,1].set_ylabel('Steps/Second')

axes[1,0].plot(steps, training_metrics['memory_used'])
axes[1,0].set_title('Memory Usage')
axes[1,0].set_xlabel('Step')
axes[1,0].set_ylabel('Bytes')

axes[1,1].plot(steps, training_metrics['tpu_utilization'])
axes[1,1].set_title('TPU Utilization')
axes[1,1].set_xlabel('Step')
axes[1,1].set_ylabel('Utilization %')

plt.tight_layout()
plt.show()

In [ ]:
# Get performance recommendations
recommendations = profiler.get_performance_recommendations()
print("\nPerformance Recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec}")

## Model Performance Analysis

Current metrics from previous analysis:
- Total Parameters: 109,529,088
- MoE Parameters: 301,991,936
- Memory Usage:
  - Activations: 96.00 MB
  - Attention: 3072.00 MB
  - KV Cache: 192.00 MB

In [ ]:
def profile_model_performance(config: Dict[str, Any]):
    """Profile model inference performance"""
    model = create_vishwamai_transformer(config)
    batch_size = 1
    seq_length = config['max_seq_len']
    
    # Create dummy input
    dummy_input = jnp.ones((batch_size, seq_length), dtype=jnp.int32)
    
    # Profile memory
    memory_profile = profiler.profile_memory_usage(
        lambda x: model.apply({'params': state.params}, x),
        {'input': dummy_input.shape}
    )
    
    return memory_profile

# Run performance profiling
perf_metrics = profile_model_performance(config['model_config'])
print("\nModel Performance Profile:")
for k, v in perf_metrics.items():
    print(f"{k}: {v/1e9:.2f} GB")

In [ ]:
def benchmark_attention(batch_size: int = 32, seq_len: int = 512):
    """Benchmark different attention implementations"""
    # Generate dummy inputs
    rng = jax.random.PRNGKey(0)
    x = jax.random.normal(rng, (batch_size, seq_len, config['model_config']['hidden_dim']))
    
    # Standard attention
    def run_std_attention():
        q = k = v = x
        scores = jnp.einsum('bqd,bkd->bqk', q, k)
        scores = scores / jnp.sqrt(config['model_config']['head_dim'])
        attn = jax.nn.softmax(scores)
        return jnp.einsum('bqk,bkd->bqd', attn, v)
    
    # Flash attention
    def run_flash_attention():
        q = k = v = x.reshape(batch_size, seq_len, 
                             config['model_config']['num_heads'], 
                             config['model_config']['head_dim'])
        return FlashAttention(
            num_heads=config['model_config']['num_heads'],
            head_dim=config['model_config']['head_dim']
        )(q, k, v)
    
    # Benchmark
    std_time = %timeit -o -n 10 -r 3 -q run_std_attention()
    flash_time = %timeit -o -n 10 -r 3 -q run_flash_attention()
    
    return {
        'standard_attention_ms': std_time.best * 1000,
        'flash_attention_ms': flash_time.best * 1000,
        'speedup': std_time.best / flash_time.best
    }

In [ ]:
# Run benchmarks
print("Running attention benchmarks...")
results_512 = benchmark_attention(seq_len=512)
results_1024 = benchmark_attention(seq_len=1024)
results_2048 = benchmark_attention(seq_len=2048)

# Plot results
seq_lens = [512, 1024, 2048]
std_times = [results_512['standard_attention_ms'],
            results_1024['standard_attention_ms'],
            results_2048['standard_attention_ms']]
flash_times = [results_512['flash_attention_ms'],
              results_1024['flash_attention_ms'],
              results_2048['flash_attention_ms']]

plt.figure(figsize=(10, 6))
plt.plot(seq_lens, std_times, 'b-', label='Standard Attention')
plt.plot(seq_lens, flash_times, 'r-', label='Flash Attention')
plt.xlabel('Sequence Length')
plt.ylabel('Time (ms)')
plt.title('Attention Performance Comparison')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nSpeedup at different sequence lengths:")
print(f"512: {results_512['speedup']:.2f}x")
print(f"1024: {results_1024['speedup']:.2f}x")
print(f"2048: {results_2048['speedup']:.2f}x")

In [ ]:
def benchmark_tpu_performance(config: Dict[str, Any]) -> Dict[str, float]:
    """Benchmark TPU performance characteristics"""
    # Initialize model
    model = create_vishwamai_transformer(config)
    
    # Setup profiler
    profiler = TPUProfiler(config)
    
    # Generate sample batch
    batch_size = 32
    seq_len = 512
    x = jnp.ones((batch_size, seq_len), dtype=jnp.int32)
    
    # Initialize model parameters
    rng = jax.random.PRNGKey(0)
    variables = model.init(rng, x)
    
    # Warmup
    with profiler.profile_region('warmup'):
        _ = model.apply(variables, x, deterministic=True)  # Fixed: Using deterministic instead of train
    
    # Benchmark forward pass
    times = []
    for _ in range(10):
        start = time.time()
        _ = model.apply(variables, x, deterministic=True)  # Fixed: Using deterministic instead of train
        jax.tree_util.tree_map(lambda x: x.block_until_ready(), _)
        times.append(time.time() - start)
    
    # Calculate metrics
    forward_latency = np.mean(times)
    tokens_per_second = (batch_size * seq_len) / forward_latency
    
    # Get TPU utilization from profiler
    metrics = profiler.get_metrics_summary()
    tpu_utilization = metrics.get('tpu_utilization_mean', 0.0)
    memory_efficiency = metrics.get('memory_efficiency_mean', 0.0)
    
    return {
        'forward_latency': forward_latency,
        'tokens_per_second': tokens_per_second,
        'tpu_utilization': tpu_utilization,
        'memory_efficiency': memory_efficiency
    }